## Update info

<u>Version1</u>: 

* First release. I tune epoch and so no.

<u>Version2</u>: 

* Change following points:

    * Used EDGE_ENHANCE filter to data.

    * Used OneCycleLR Scheduler.
    
<u>Version3</u>: 

* Added support for new datasets; removed EDGE_ENHANCE filter once and for all.

------------------------

I create train and inference notebook for Unet++ of [segmentation_models.pytorch](https://github.com/qubvel/segmentation_models.pytorch).

I use only pytorch for framework.

The accuracy of the model is going to be tuned and improved in the future.

I published this notebook for our reference as an example implementation using only pytorch.

I refered following two great notebooks for training and inference.

- https://www.kaggle.com/iafoss/hubmap-pytorch-fast-ai-starter

- https://www.kaggle.com/curiosity806/hubmap-use-catalyst-smp-and-albumentations

And refered following great notebook for dice loss.

- https://www.kaggle.com/bigironsphere/loss-function-library-keras-pytorch

Kindly upvote and appreciate the original work.

## Load libraries

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import gc
import os
import random
import time
import warnings
warnings.simplefilter("ignore")

#import pdb
#import zipfile
#import pydicom
from albumentations import *
from albumentations.pytorch import ToTensor
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold
import tifffile as tiff
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, sampler
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

## Dataset

In [ ]:
!mkdir ./masks
!mkdir ./train

!unzip ../input/256x256-images/masks.zip -d ./masks
!unzip ../input/256x256-images/train.zip -d ./train

##  Set parameters

In [ ]:
def set_seed(seed=2**3):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(121)

To save time, the number of "folds" is smaller. 
These days, depending on the model, I think it's more common to use 5 ~ 10.

In [ ]:
fold = 0
nfolds = 5
reduce = 4
sz = 256

BATCH_SIZE = 16
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 15
NUM_WORKERS = 4
SEED = 2020
TH = 0.39  #threshold for positive predictions

DATA = '../input/hubmap-kidney-segmentation/test/'
LABELS = '../input/hubmap-kidney-segmentation/train.csv'
MASKS = './masks/'
TRAIN = './train/'
df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')

## Util functions

In [ ]:
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with bug fix
def rle_encode_less_memory(img):
    #watch out for the bug
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

## Dataset

In [ ]:
# https://www.kaggle.com/iafoss/256x256-images
mean = np.array([0.65527532, 0.49901106, 0.69247992] )
std = np.array([0.25565283, 0.31975344, 0.21533712])

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

class HuBMAPDataset(Dataset):
    def __init__(self, fold=fold, train=True, tfms=None):
        ids = pd.read_csv(LABELS).id.values
        kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)
        if self.tfms is not None:
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        return img2tensor((img/255.0 - mean)/std),img2tensor(mask)

In [ ]:
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            HueSaturationValue(10,15,10),
            CLAHE(clip_limit=2),
            RandomBrightnessContrast(),            
        ], p=0.3),
    ], p=p)

In [ ]:
#example of train images with masks
ds = HuBMAPDataset(tfms=get_aug())
dl = DataLoader(ds,batch_size=64,shuffle=False,num_workers=NUM_WORKERS)
imgs,masks = next(iter(dl))

plt.figure(figsize=(16,16))
for i,(img,mask) in enumerate(zip(imgs,masks)):
    img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
    plt.subplot(8,8,i+1)
    plt.imshow(img,vmin=0,vmax=255)
    plt.imshow(mask.squeeze().numpy(), alpha=0.2)
    plt.axis('off')
    plt.subplots_adjust(wspace=None, hspace=None)
    
del ds,dl,imgs,masks

## Model

In [ ]:
def get_UnetPlusPlus():
    model =  smp.UnetPlusPlus(
                 encoder_name='efficientnet-b3',
                 encoder_weights='imagenet',
                 in_channels=3,
                 classes=1)
    return model

## DiceLoss

Note that this loss represents 1 - DiceLoss.

In [ ]:
#https://www.kaggle.com/bigironsphere/loss-function-library-keras-pytorch
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

## Training

In [ ]:
cv_score = 0
for fold in range(nfolds):
    ds_t = HuBMAPDataset(fold=fold, train=True, tfms=get_aug())
    ds_v = HuBMAPDataset(fold=fold, train=False)
    dataloader_t = torch.utils.data.DataLoader(ds_t,batch_size=BATCH_SIZE, shuffle=False,num_workers=NUM_WORKERS)
    dataloader_v = torch.utils.data.DataLoader(ds_t,batch_size=BATCH_SIZE, shuffle=False,num_workers=NUM_WORKERS)
    model = get_UnetPlusPlus().to(DEVICE)
    
    optimizer = torch.optim.Adam([
        {'params': model.decoder.parameters(), 'lr': 1e-3}, 
        {'params': model.encoder.parameters(), 'lr': 1e-3},  
    ])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(dataloader_t))
    
    diceloss = DiceLoss()
    
    print(f"########FOLD: {fold}##############")
    
    for epoch in tqdm(range(EPOCHS)):
        ###Train
        model.train()
        train_loss = 0
    
        for data in dataloader_t:
            optimizer.zero_grad()
            img, mask = data
            img = img.to(DEVICE)
            mask = mask.to(DEVICE)
        
            outputs = model(img)
    
            loss = diceloss(outputs, mask)
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
        train_loss /= len(dataloader_t)
        
        print(f"FOLD: {fold}, EPOCH: {epoch + 1}, train_loss: {train_loss}")
        
        ###Validation
        model.eval()
        valid_loss = 0
        
        for data in dataloader_v:
            img, mask = data
            img = img.to(DEVICE)
            mask = mask.to(DEVICE)
        
            outputs = model(img)
    
            loss = diceloss(outputs, mask)
        
            valid_loss += loss.item()
        valid_loss /= len(dataloader_v)
        
        print(f"FOLD: {fold}, EPOCH: {epoch + 1}, valid_loss: {valid_loss}")
        
        
    ###Save model
    torch.save(model.state_dict(), f"FOLD{fold}_.pth")
    
    cv_score += valid_loss
    
cv_score = cv_score/nfolds

In [ ]:
print(f"CV score is: {cv_score}")

In [ ]:
!rm -r ./masks
!rm -r ./train